In [ ]:
import sqlite3 as sql
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas_datareader import data as web 


import os
notebook_directory = os.getcwd()
print(notebook_directory)


In [ ]:




def load_data():
    # Load the data from SQL database
    conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db') 
    df_sum = pd.read_sql_query("SELECT * FROM PORTFOLIO", conn)
    conn.close

    # Data preparation
    df_sum['TotalAmount'] = df_sum.groupby('Ticker')['Amount'].transform('sum')
    df_sum.rename(columns={'TotalAmount':'Quantity'}, inplace=True)

    pd.set_option('display.float_format', '{:.6f}'.format)
    df_prepeared = df_sum[['Ticker', 'Quantity']].drop_duplicates()
    tickers_to_drop = ['VTX', 'CITY', 'IONX']# Drop a few rows for now due to an issue with the market maker. In the future, try using API requests with another financial aggregator.
    df_prepeared = df_prepeared[~df_prepeared['Ticker'].isin(tickers_to_drop)]

    return df_prepeared, df_sum

In [ ]:
df_prepeared, df_sum = load_data()

In [ ]:
df_prepeared = df_prepeared.drop("Quantity", axis=1)
df_prepeared = df_prepeared.drop_duplicates(subset='Ticker')
df_prepeared = df_prepeared.reset_index(drop=True)

print(df_prepeared)


In [ ]:
df_prepeared = df_prepeared.reset_index(drop=True)
print(df_prepeared)

In [ ]:
def insert_data(conn, df_prepeared):
    conn.execute('''INSERT INTO TOKENSINFO (Ticker, Information) VALUES (?, ?)''', (df_prepeared, ''))
    conn.commit()



In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')


for index, row in df_prepeared.iterrows():
    ticker = row['Ticker']
    insert_data(conn, ticker)
conn.commit()
conn.close()